In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import csv
from torch.autograd import Variable

images = np.load("D:/work/JHUschoolStuff/machinelearning/project1/cs475_project_data/images.npy")
labels = np.load("D:/work/JHUschoolStuff/machinelearning/project1/cs475_project_data/labels.npy")
test = np.load("D:/work/JHUschoolStuff/machinelearning/project1/cs475_project_data/test_images.npy")
height = images.shape[1]
width = images.shape[2]
size = height*width
images = (images - images.mean()) / images.std()
data = images.reshape(images.shape[0],size)
test_data = test.reshape(test.shape[0], size)
test_data = (test_data - test_data.mean()) / test_data.std()
training = 40000
validation = 45000
testing = 50000
apple_label = 0
rate = 0.6
iterations = 3
npw = np.random.rand(5, size)
w = Variable(torch.from_numpy(npw).float(), requires_grad=True)
w_out = Variable(torch.FloatTensor(5).uniform_(0,1).float(), requires_grad=True)
accuracies = np.zeros(400)
val_accuracies = np.zeros(400)
w, w_out = fit(w, w_out, data, rate, iterations, accuracies, val_accuracies)
y_hat = prediction(w, w_out, data)


Not overfitting
Given that our validation data is giving us the same level of accuracy that our training data is giving, it is unlikely that we are overfitting our training data.

In [16]:
def fit(w, w_out, data, rate, iterations, accuracies, val_accuracies):
    for k in range(0, iterations):
        count = 0
        count_2 = 0
        for i in range(0, 40000):
            x = torch.from_numpy(data[i]).float()
            X_= Variable(x).unsqueeze(1)
            val = 1
            if labels[i] != 0:
                val = -1
            multi = torch.clamp(torch.mm(w, X_), -500, 500).float()
            activated = torch.sigmoid(multi)
            onedAct = activated.squeeze(1)
            out = torch.clamp(w_out.dot(onedAct), -500, 500).float()
            loss = torch.log(1 + torch.exp(-1*val*out))
            loss.backward()

            w_out.data = w_out.data - rate*w_out.grad.data
            w.data = w.data - rate*w.grad.data
            w.grad.data.zero_()
            w_out.grad.data.zero_()

            if i % 100 == 0:
                y_hat = np.zeros(1000)
                y = np.zeros(1000)
                for j in range(0, 1000):
                    s = np.random.randint(0,40000)
                    dat = torch.from_numpy(data[s]).float()
                    dat_= Variable(dat).unsqueeze(1)
                    hidden = torch.mm(w, dat_).float()
                    hidden_activated = torch.sigmoid(hidden)
                    oneD = hidden_activated.squeeze(1)
                    output = w_out.dot(oneD)
                    activate_out = torch.sigmoid(output).float()
                    if activate_out.data[0] < 0.5:
                        y_hat[j] = 0.0
                    else:
                        y_hat[j] = 1.0
                    if labels[s] == 0:
                        y[j] = 1.0
                    else:
                        y[j] = 0.0

                accuracies[count] = accuracy(y, y_hat)
                count = count + 1

                for l in range(0, 1000):
                    s = np.random.randint(40000,45000)
                    dat = torch.from_numpy(data[s]).float()
                    dat_= Variable(dat).unsqueeze(1)
                    hidden = torch.mm(w, dat_).float()
                    hidden_activated = torch.sigmoid(hidden)
                    oneD = hidden_activated.squeeze(1)
                    output = w_out.dot(oneD)
                    activate_out = torch.sigmoid(output).float()
                    if activate_out.data[0] < 0.5:
                        y_hat[l] = 0.0
                    else:
                        y_hat[l] = 1.0
                    if labels[s] == 0:
                        y[l] = 1.0
                    else:
                        y[l] = 0.0
                val_accuracies[count_2] = accuracy(y, y_hat)
                count_2 = count_2 + 1
    print(accuracies)
    plt.figure(200)
    plt.title('Training Accuracy')
    plt.xlabel('Iteration')
    plt.ylabel('Accuracy')
    plt.plot(accuracies, 'b')
    plt.show()
    plt.figure(300)
    plt.title('Validation Accuracy')
    plt.xlabel('Iteration')
    plt.ylabel('Accuracy')
    plt.plot(val_accuracies, 'b')
    plt.show()
    return w, w_out


In [3]:
def prediction(w, w_out, data):
    y_hat = np.zeros(5000)
    for j in range(45000, 50000):
        dat = torch.from_numpy(data[j]).float()
        dat_= Variable(dat).unsqueeze(1)
        hidden = torch.mm(w, dat_).float()
        hidden_activated = torch.sigmoid(hidden)
        oneD = hidden_activated.squeeze(1)
        output = w_out.dot(oneD)
        activate_out = torch.sigmoid(output).float()
        if activate_out.data[0] < 0.5:
            y_hat[j - 45000] = 0.0
        else:
            y_hat[j - 45000] = 1.0
    return y_hat
    

In [4]:
def accuracy(y, y_hat):
    return (y == y_hat).astype(np.float).mean()

In [39]:
with open('jzhan127.csv', 'w', newline='') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerow(['id', 'label'])
    for i in range(0, 5000):
        dat = torch.from_numpy(test_data[i]).float()
        dat_= Variable(dat).unsqueeze(1)
        hidden = torch.mm(w, dat_).float()
        hidden_activated = torch.sigmoid(hidden)
        oneD = hidden_activated.squeeze(1)
        output = w_out.dot(oneD)
        activate_out = torch.sigmoid(output).float()
        if activate_out.data[0] < 0.5:
            filewriter.writerow([i, '1'])
        else:
            filewriter.writerow([i, '0'])

Kaggle submission: jzhan127.csv